In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

C:\Users\test.DESKTOP-ODO5P7A\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 3276 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:3276/status,
Dashboard: http://127.0.0.1:3276/status,Workers: 1
Total threads: 4,Total memory: 63.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:3277,Workers: 1
Dashboard: http://127.0.0.1:3276/status,Total threads: 4
Started: Just now,Total memory: 63.92 GiB
Comm: tcp://127.0.0.1:3286,Total threads: 4
Dashboard: http://127.0.0.1:3287/status,Memory: 63.92 GiB
Nanny: tcp://127.0.0.1:3280,


In [2]:
import random
import time

def inc(x):
    time.sleep(random.random())
    return x + 1
def double(x):
    time.sleep(random.random())
    return 2 * x
def add(x, y):
    time.sleep(random.random())
    return x + y

In [3]:
inc(1)

2

In [4]:
future = client.submit(inc, 1)
# returns immediately with pending future
future

<Future: pending, key: inc-0b4a877b2a9917cb3aaa0e25790a1f9c>

In [5]:
future
# scheduler and client talk constantl

<Future: finished, type: int, key: inc-0b4a877b2a9917cb3aaa0e25790a1f9c>

In [6]:
future.result()

2

In [7]:
x = client.submit(inc, 1)
y = client.submit(double, 2)
z = client.submit(add, x, y)
z

<Future: pending, key: add-950f83a08ff633df873deb72d3515ff4>

In [8]:
z.result()

6

In [14]:
zs= []


for i in range(256):
    x = client.submit(inc, i)
    # x = inc(i)
    y = client.submit(double, x)
    # y = inc(x)
    z = client.submit(add, x, y)
    # z = inc(y)
    zs.append(z)

In [15]:
total = client.submit(sum, zs)

In [16]:
client.cluster.scale(10)

In [17]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = client.submit(add, L[i], L[i + 1])
    # add neighbors
        new_L.append(future)
    L = new_L

In [18]:
del future, L, new_L, total

In [20]:
from dask.distributed import as_completed

zs = client.map(inc, zs)
seq = as_completed(zs)
while seq.count() > 1:
    # at least two futures left
    a = next(seq)
    b = next(seq)
    new = client.submit(add, a, b, priority=1)
# add them together
    seq.add(new)